In [1]:
import os

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/home/husein/t5/prepare/mesolitica-tpu.json'
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
import tensorflow as tf
from pegasus import transformer

In [3]:
vocab_size = 32128
hidden_size = 512
filter_size = 3072
num_encoder_layers = 6
num_decoder_layers = 6
num_heads = 8
label_smoothing = 0.0
dropout = 0.1

In [4]:
model = transformer.TransformerEncoderDecoderModel(vocab_size, hidden_size,
                                                   filter_size, num_heads,
                                                   num_encoder_layers,
                                                   num_decoder_layers,
                                                   label_smoothing, dropout)

In [5]:
X = tf.placeholder(tf.int64, (None, None))
top_p = tf.placeholder(tf.float32, None)
outputs = model.predict(
  {
      "inputs": X,
  }, tf.shape(X)[1], 1)




Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor



In [6]:
import sentencepiece as spm

vocab = '/home/husein/b2b/sp10m.cased.t5.model'
sp = spm.SentencePieceProcessor()
sp.Load(vocab)

class Encoder:
    def __init__(self, sp):
        self.sp = sp
        self.vocab_size = sp.GetPieceSize() + 100
    
    def encode(self, s):
        return self.sp.EncodeAsIds(s)
    
    def decode(self, ids, strip_extraneous=False):
        return self.sp.DecodeIds(list(ids))
    
encoder = Encoder(sp)

In [7]:
import tensorflow as tf
import os

# ckpt_path = 'base/model.ckpt-300000'
ckpt_path = tf.train.latest_checkpoint('gs://mesolitica-tpu-general/pegasus-small-multitasks')
ckpt_path

'gs://mesolitica-tpu-general/pegasus-small-multitasks/model.ckpt-550000'

In [8]:
sess = tf.InteractiveSession()

In [9]:
var_lists = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
saver = tf.train.Saver(var_list = var_lists)
saver.restore(sess, ckpt_path)

INFO:tensorflow:Restoring parameters from gs://mesolitica-tpu-general/pegasus-small-multitasks/model.ckpt-550000


In [10]:
import re
from unidecode import unidecode

def cleaning(string):
    return re.sub(r'[ ]+', ' ', unidecode(string)).strip()

In [11]:
string = """
SHAH ALAM - Langkah Tun Dr Mahathir Mohamad yang mahu menubuhkan parti baharu sekali lagi dilihat akan memecahbelahkan orang Melayu, kata bekas pegawai khas Datuk Seri Najib Tun Razak, Isham Jalil.

Dakwa beliau, orang Melayu akan menjadi semakin lemah akibat tindakan bekas Perdana Menteri itu.

"Lebih dua puluh tahun dahulu, Dr Mahathir telah memecahbelahkan orang-orang Melayu dengan penubuhan Parti Keadilan Rakyat (PKR) oleh Datuk Seri Anwar Ibrahim akibat pergaduhan Dr Mahathir dengan beliau.

"Lima tahun lalu juga, Dr Mahathir telah memecahbelahkan orang-orang Melayu melalui penubuhan Bersatu untuk menjatuhkan Najib kerana tidak membantu Mukhriz dalam pemilihan UMNO serta tidak membantu kroni-kroni beliau untuk menyambung kontrak dan konsesi kerajaan yang diperoleh selama berpuluh tahun sebelum itu.

"Ini tidak akan berhenti sehinggalah Mukhriz menjadi Perdana Menteri dan kepentingan peribadi serta kroni-kroninya dipenuhi," katanya dalam satu kenyataan hari ini.

Justeru katanya, Ahli Parlimen Langkawi itu layak dilabelkan sebagai 'Bapa Pemecahbelah Orang-orang Melayu'.

"Parti baharu Dr Mahathir ini dikhabarkan yang terdiri daripada serpihan ahli-ahli Bersatu dalam kalangan orang-orang Melayu.

"Sebab jika ada banyak parti Melayu, orang-orang Melayu berkemungkinan besar akan lebih bertelagah dan berpecah belah," katanya.

Isham yang juga Ketua Penerangan Barisan Nasional (BN) Selangor berkata, musuh Melayu dan DAP kini paling beruntung dan bersorak dengan tindakan Dr Mahathir ini.

"Jika masih ada lagi orang-orang Melayu yang masih tidak sedar dan masih mengikut Dr Mahathir, saya tidak tahu nak kata apa.

"Kepada Dr Mahathir dan pengikut-pengikutnya ini, daripada mereka menjadi musuh dalam selimut untuk orang-orang Melayu, mungkin lebih baik mereka terus masuk DAP secara terang-terangan seperti segelintir Melayu DAP yang lain.

"Dengan ini mereka tidak perlu menyorok atau berselindung lagi tentang perjuangan sebenar mereka," katanya.
"""

In [12]:
string1 = """
KUALA LUMPUR: Dakwaan Datuk Seri Anwar Ibrahim semalam, bahawa beliau mendapat sokongan 'kukuh dan meyakinkan' untuk mengambil alih tampuk pemerintahan, menimbulkan pelbagai persoalan sejauh manakah beliau mampu menggugat kerajaan Perikatan Nasional (PN).

Kerajaan PN akan terus memerintah selagi Presiden Parti Keadilan Rakyat (PKR) itu dapat membuktikan kepada Yang di-Pertuan Agong dan Dewan Rakyat yang beliau benar-benar mempunyai angka keramat untuk mengambil alih tampuk pemerintahan kerajaan PN.

Bagi pakar geostrategi. Azmi Hassan, walau apapun percaturan Anwar, kata putus akhir mengenai sebarang perubahan pada tampuk pemerintahan bukan terletak di tangan beliau tetapi di tangan Yang di-Pertuan Agong Al-Sultan Abdullah Ri'ayatuddin Al-Mustafa Billah Shah.

Beliau berkata Agong boleh memutuskan peralihan kuasa dilakukan tanpa pembubaran Parlimen atau pilihan raya umum jika Parlimen dibubarkan.

"Memang benar jawatan Perdana Menteri adalah bergantung kepada sokongan yang diperoleh melalui Dewan Rakyat. Jika benar dakwaan yang beliau telah mendapat sokongan majoriti, ini bermakna Perdana Menteri, Tan Sri Muhyiddin Yassin sudah kehilangan sokongan Ahli Dewan Rakyat," katanya.

Mengenai desas-desus bahawa Anwar mempunyai sokongan Ahli Parlimen UMNO, Azmi berkata beliau tidak pasti tentang perkara itu memandangkan ia akan menimbulkan kekecewaan akar umbi UMNO kerana mana mungkin parti itu boleh bekerjasama dengan DAP yang selama ini dianggap seteru.

"Perlu diingat mengapa Ahli Parlimen UMNO sokong Muhyiddin adalah untuk mengelak DAP dalam kerajaan.

"Oleh kerana DAP sudah menyatakan sokongan kepada Anwar, maka agak pasti DAP akan turut serta dalam kerajaan Anwar. Ini sudah tentu menyukarkan penerimaan akar umbi UMNO terhadap keputusan Ahli Parlimen UMNO sokong Anwar," katanya.

Presiden UMNO, Datuk Seri Dr Ahmad Zahid Hamidi dalam kenyataan semalam, mengesahkan menerima maklumat terdapat ramai ahli Parlimen UMNO dan Barisan Nasional (BN) menyatakan sokongan terhadap Anwar dan beliau menghormati pendirian mereka itu.

Sementara itu, Timbalan Pengarah Institut Kajian Etnik (KITA) Universiti Kebangsaan Malaysia (UKM), Profesor Madya Dr Kartini Aboo Talib berkata, masyarakat secara umumnya tidak boleh menerima bulat-bulat dakwaan Anwar mendapat sokongan membentuk kerajaan baharu.

Ini kerana dakwaan itu perlu dibuktikan terlebih dahulu dengan melihat kepada akuan bersumpah (SD) yang ditandatangani barisan Ahli Parlimen.

"Pada saya apabila Anwar mengatakan telah mendapat sokongan majoriti, itu hanyalah kata-kata. Perlu dibuktikan dengan beberapa cara termasuk melihat kepada SD atau pengumuman secara jelas daripada Ahli Parlimen. Jadi, kita tidak perlu menerima bulat-bulat kenyataan beliau lagi," katanya.

Bagaimanapun, beliau mengakui pengumuman dibuat Anwar itu mampu mempengaruhi proses Pilihan Raya Negeri Sabah Sabtu ini memandangkan pengundi di negeri itu perlu memilih gabungan yang disenangi kerajaan Persekutuan demi masa depan Sabah.

"Hubungan antara Persekutuan dan negeri memberi impak besar dalam pembangunan di Sabah selain melihat bagaimana keakuran Persekutuan berhubung Perjanjian Malaysia 1963 (MA63)," katanya.

Sementara itu, belum ada pihak dalam PN yang menyatakan sokongan kepada Anwar. - BERNAMA
"""

In [13]:
encoded = encoder.encode(f'tajuk: {cleaning(string)}') + [1]
f = sess.run(outputs, feed_dict = {X: [encoded], top_p: 0.8})
encoder.decode(f['outputs'][0].tolist())

'Tun Dr Mahathir mahu menubuhkan parti baharu'

In [14]:
encoded = encoder.encode(f'tajuk: {cleaning(string1)}') + [1]
f = sess.run(outputs, feed_dict = {X: [encoded]})
encoder.decode(f['outputs'][0].tolist())

'Dakwaan Datuk Seri Anwar Ibrahim'